# Lab 1
## AirBnB Listings for Los Angeles Area
### Jason McDonald, Miguel Bonilla, Zachary Bunn

The site we got the data from is about the wastefulness of airbnb, but I think for our class it would be better (easier) to focus on the busness opportunity.
So, say, building a model that predicts daily rental listing price, using an error metric to measure the effectiveness of the predictions.
This would definitely be more tangible than anything from the side of the wasted housing space which would be harder to measure.

### Business Understanding

In [1]:
## load pandas and import dataset from GitHub
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px

df = pd.read_csv("https://media.githubusercontent.com/media/boneeyah/DS7331_Group/main/Data_Files/airbnb_los_angeles.csv")

In [2]:
# verify data loaded properly
df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,45392,https://www.airbnb.com/rooms/45392,20220606014052,2022-06-06,Cute Home in Mount Washington,<b>The space</b><br />Cute house in Mount Wash...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,201514,https://www.airbnb.com/users/show/201514,...,4.96,4.58,4.85,NaN,f,1,0,1,0,0.20
1,45417,https://www.airbnb.com/rooms/45417,20220606014052,2022-06-06,Silver Lake Cottage Oasis- Private Terrace- Th...,"Ideal for an extended stay, our Cottage perfec...",Walk just a couple of blocks to reach all the ...,https://a0.muscache.com/pictures/monet/Select-...,50231247,https://www.airbnb.com/users/show/50231247,...,4.98,4.91,4.82,NaN,f,4,4,0,0,1.09
2,5728,https://www.airbnb.com/rooms/5728,20220606014052,2022-06-06,Tiny Home in Artistic Oasis near Venice and LAX,Our home is located near Venice Beach without ...,Our neighborhood is very quiet and save. There...,https://a0.muscache.com/pictures/7a29d275-f293...,9171,https://www.airbnb.com/users/show/9171,...,4.92,4.80,4.70,HSR19-002149,f,3,0,3,0,2.00
3,5729,https://www.airbnb.com/rooms/5729,20220606014052,2022-06-06,Zen Room with Floating Bed near Venice and LAX,Our home is located near Venice Beach without ...,Our neighborhood is very quiet and save. There...,https://a0.muscache.com/pictures/f48e3ea8-2075...,9171,https://www.airbnb.com/users/show/9171,...,4.89,4.77,4.71,HSR19-002149,f,3,0,3,0,1.52
4,109,https://www.airbnb.com/rooms/109,20220606014052,2022-06-06,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...",NaN,https://a0.muscache.com/pictures/4321499/1da98...,521,https://www.airbnb.com/users/show/521,...,4.00,5.00,4.00,NaN,f,1,1,0,0,0.02


In [3]:
# delete this later on
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            42041 non-null  int64  
 1   listing_url                                   42041 non-null  object 
 2   scrape_id                                     42041 non-null  int64  
 3   last_scraped                                  42041 non-null  object 
 4   name                                          42039 non-null  object 
 5   description                                   41102 non-null  object 
 6   neighborhood_overview                         25110 non-null  object 
 7   picture_url                                   42041 non-null  object 
 8   host_id                                       42041 non-null  int64  
 9   host_url                                      42041 non-null 

In [4]:
## delete this later on
## this can be deleted, was checking for difference between the two
print(df.room_type.unique())
df.property_type.unique()

## not sure if we should keep both, they're sort of related, but sufficiently different

['Private room' 'Entire home/apt' 'Shared room' 'Hotel room']


array(['Private room in home', 'Entire bungalow',
       'Private room in tiny home', 'Private room in guesthouse',
       'Entire condo', 'Room in hotel', 'Private room in rental unit',
       'Entire guesthouse', 'Entire home', 'Entire rental unit',
       'Entire guest suite', 'Private room in loft', 'Entire townhouse',
       'Entire cottage', 'Entire villa', 'Private room',
       'Shared room in rental unit', 'Room in boutique hotel',
       'Private room in townhouse', 'Private room in hostel', 'Camper/RV',
       'Private room in bed and breakfast', 'Farm stay', 'Entire cabin',
       'Entire loft', 'Private room in farm stay',
       'Private room in condo', 'Private room in bungalow',
       'Private room in guest suite', 'Shared room in home',
       'Shared room in hostel', 'Shared room in villa', 'Entire place',
       'Room in aparthotel', 'Private room in treehouse', 'Treehouse',
       'Yurt', 'Private room in villa', 'Private room in cottage',
       'Private room in c

In [5]:
## removing attributes whch have no value to us
### keeping longitude and latitude in case we wanted to do a sort of heat map for airbnb hotspots for the exceptional points, if not we can remove
### neighborhood, neighborhood_cleansed, and neighborhood_group_cleansed variations of same, but cleansed seems to be better to use, I'm guessing it has imputed values
### for some reason, all listings have NaN for bathrooms, so removing it
for col in [
    'listing_url','scrape_id','last_scraped','description','neighborhood_overview','picture_url','host_url','host_about','host_response_time','host_response_rate','host_acceptance_rate',
    'host_thumbnail_url','host_picture_url','host_verifications','host_has_profile_pic','bathroom_text','host_listings_count','host_neighbourhood','bathrooms','minimum_minimum_nights',
    'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','availability_30','availability_60',
    'availability_90','availability_365','calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d','review_scores_accuracy','review_scores_communication','review_scores_cleanliness',
    'review_scores_checkin','review_scores_value','review_scores_location','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms','reviews_per_month','neighbourhood','neighbourhood_group_cleansed', 'first_review','last_review','minimum_nights','maximum_nights','license'
]:
    if col in df:
        del df[col]

In [6]:
### delete this cell later on
df.info()
### I'm thinking we could drop minimum and maximum number of night stays, it's basically shortest and longest stay at the rental, that would put us at 25 attributes unless we readd some

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              42041 non-null  int64  
 1   name                            42039 non-null  object 
 2   host_id                         42041 non-null  int64  
 3   host_name                       41858 non-null  object 
 4   host_since                      41858 non-null  object 
 5   host_location                   41780 non-null  object 
 6   host_is_superhost               41858 non-null  object 
 7   host_total_listings_count       41858 non-null  float64
 8   host_identity_verified          41858 non-null  object 
 9   neighbourhood_cleansed          42041 non-null  object 
 10  latitude                        42041 non-null  float64
 11  longitude                       42041 non-null  float64
 12  property_type                   

In [7]:
df.price.unique()

array(['$60.00', '$135.00', '$50.00', ..., '$1,320.00', '$743.00',
       '$1,360.00'], dtype=object)

### Data Meaning Type

The data set consisted of 74 attributes, however, some of these attributes were not useful for the scope of our project (URLs and other variables), so the data set was reduced to 25 variables.

### Data Quality

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              42041 non-null  int64  
 1   name                            42039 non-null  object 
 2   host_id                         42041 non-null  int64  
 3   host_name                       41858 non-null  object 
 4   host_since                      41858 non-null  object 
 5   host_location                   41780 non-null  object 
 6   host_is_superhost               41858 non-null  object 
 7   host_total_listings_count       41858 non-null  float64
 8   host_identity_verified          41858 non-null  object 
 9   neighbourhood_cleansed          42041 non-null  object 
 10  latitude                        42041 non-null  float64
 11  longitude                       42041 non-null  float64
 12  property_type                   

listing name, host_name, host_since, host_location, host_is_superhost, host_total_listings_count, host_identity_verified, bathrooms_text, bedrooms, beds and review_scores_rating all have missing values.
With review scores having the largest number of missing values. 

In [9]:
## drop results with no review scores, we can redefine the scope of the project to only listings with review scores available or something like that
df = df[~df.review_scores_rating.isnull() & ~df.beds.isnull() & ~df.host_location.isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31871 entries, 0 to 42000
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              31871 non-null  int64  
 1   name                            31871 non-null  object 
 2   host_id                         31871 non-null  int64  
 3   host_name                       31871 non-null  object 
 4   host_since                      31871 non-null  object 
 5   host_location                   31871 non-null  object 
 6   host_is_superhost               31871 non-null  object 
 7   host_total_listings_count       31871 non-null  float64
 8   host_identity_verified          31871 non-null  object 
 9   neighbourhood_cleansed          31871 non-null  object 
 10  latitude                        31871 non-null  float64
 11  longitude                       31871 non-null  float64
 12  property_type                   

In [10]:
## there are more bedrooms NAs than beds, I'm thinking we could impute #bedrooms based on the median bedrooms for listings with that same number of beds
df_grouped = df.groupby(by = ['beds'])
df_grouped[['bedrooms']].describe()

bedrooms                                                     
        count       mean       std   min    25%   50%    75%   max
beds                                                              
1.0   12043.0   1.014033  0.145413   1.0   1.00   1.0   1.00   5.0
2.0    7044.0   1.509086  0.524891   1.0   1.00   1.0   2.00   4.0
3.0    4057.0   2.325857  0.698550   1.0   2.00   2.0   3.00   6.0
4.0    2732.0   2.905930  0.902583   1.0   2.00   3.0   4.00   6.0
5.0    1337.0   3.498130  1.009775   1.0   3.00   4.0   4.00   6.0
6.0     806.0   3.818859  1.174022   1.0   3.00   4.0   5.00   7.0
7.0     403.0   4.203474  1.280659   1.0   3.00   4.0   5.00   7.0
8.0     252.0   4.583333  1.415973   1.0   4.00   4.0   5.00   8.0
9.0     108.0   4.842593  1.254242   3.0   4.00   5.0   6.00   9.0
10.0     69.0   4.811594  1.620415   1.0   4.00   5.0   5.00  10.0
11.0     43.0   5.465116  1.386136   3.0   4.00   5.0   6.00  10.0
12.0     24.0   6.083333  1.863066   4.0   5.00   6.0   7.00  11.0
13.0     13.0   5.307692  1.377474   4.0   4.00   5.0   7.00   7.0
14.0     12.0   6.166667  2.329000   1.0   4.75   7.0   7.25   9.0
15.0      4.0   7.250000  1.500000   6.0   6.00   7.0   8.25   9.0
16.0      9.0   4.888889  2.713137   1.0   4.00   5.0   6.00   9.0
17.0      1.0  10.000000       NaN  10.0  10.00  10.0  10.00  10.0
18.0      3.0   6.000000  4.582576   1.0   4.00   7.0   8.50  10.0
19.0      2.0   5.000000  5.656854   1.0   3.00   5.0   7.00   9.0
20.0      2.0   4.000000  4.242641   1.0   2.50   4.0   5.50   7.0
21.0      2.0   9.500000  0.707107   9.0   9.25   9.5   9.75  10.0
23.0      1.0   6.000000       NaN   6.0   6.00   6.0   6.00   6.0
30.0      1.0  17.000000       NaN  17.0  17.00  17.0  17.00  17.0

In [11]:
df_imputed = df_grouped[['beds','bedrooms']].transform(lambda grp: grp.fillna(grp.median()))
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31871 entries, 0 to 42000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   beds      31871 non-null  float64
 1   bedrooms  31871 non-null  float64
dtypes: float64(2)
memory usage: 747.0 KB


In [12]:
## imputing missing bedroom number with the median
df['imputed']=df_imputed[['bedrooms']]
## verifying that missing values where imputed with the median
df[['beds','bedrooms','imputed']][df.bedrooms.isnull()]

,beds,bedrooms,imputed
17,3.0,NaN,2.0
21,1.0,NaN,1.0
28,1.0,NaN,1.0
33,2.0,NaN,1.0
50,1.0,NaN,1.0
...,...,...,...
40952,2.0,NaN,1.0
40972,1.0,NaN,1.0
41221,1.0,NaN,1.0
41237,1.0,NaN,1.0


In [13]:
## this was just another check that imputed had 0 null values, we can probably delete this cell
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31871 entries, 0 to 42000
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              31871 non-null  int64  
 1   name                            31871 non-null  object 
 2   host_id                         31871 non-null  int64  
 3   host_name                       31871 non-null  object 
 4   host_since                      31871 non-null  object 
 5   host_location                   31871 non-null  object 
 6   host_is_superhost               31871 non-null  object 
 7   host_total_listings_count       31871 non-null  float64
 8   host_identity_verified          31871 non-null  object 
 9   neighbourhood_cleansed          31871 non-null  object 
 10  latitude                        31871 non-null  float64
 11  longitude                       31871 non-null  float64
 12  property_type                   

In [14]:
df['bedrooms'] = df['imputed']
del df['imputed']

In [15]:
## checking to see that all missing values have been handled
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31871 entries, 0 to 42000
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              31871 non-null  int64  
 1   name                            31871 non-null  object 
 2   host_id                         31871 non-null  int64  
 3   host_name                       31871 non-null  object 
 4   host_since                      31871 non-null  object 
 5   host_location                   31871 non-null  object 
 6   host_is_superhost               31871 non-null  object 
 7   host_total_listings_count       31871 non-null  float64
 8   host_identity_verified          31871 non-null  object 
 9   neighbourhood_cleansed          31871 non-null  object 
 10  latitude                        31871 non-null  float64
 11  longitude                       31871 non-null  float64
 12  property_type                   

### Simple Statistics

In [16]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31871 entries, 0 to 42000
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              31871 non-null  int64  
 1   name                            31871 non-null  object 
 2   host_id                         31871 non-null  int64  
 3   host_name                       31871 non-null  object 
 4   host_since                      31871 non-null  object 
 5   host_location                   31871 non-null  object 
 6   host_is_superhost               31871 non-null  object 
 7   host_total_listings_count       31871 non-null  float64
 8   host_identity_verified          31871 non-null  object 
 9   neighbourhood_cleansed          31871 non-null  object 
 10  latitude                        31871 non-null  float64
 11  longitude                       31871 non-null  float64
 12  property_type                   

,id,host_id,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,number_of_reviews,review_scores_rating,calculated_host_listings_count
count,3.187100e+04,3.187100e+04,31871.000000,31871.000000,31871.000000,31871.000000,31871.000000,31871.000000,31871.000000,31871.000000,31871.000000
mean,5.866474e+16,1.299227e+08,37.081077,33.989698,-118.237545,4.068871,1.738822,2.277525,47.042452,4.702408,13.238493
std,1.759227e+17,1.341366e+08,256.723524,0.191295,0.232974,2.834886,1.124086,1.745426,80.005845,0.612053,44.518969
min,1.090000e+02,5.210000e+02,0.000000,33.338650,-118.961880,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,1.959408e+07,1.979463e+07,1.000000,33.895475,-118.394550,2.000000,1.000000,1.000000,4.000000,4.670000,1.000000
50%,3.733899e+07,7.581372e+07,2.000000,34.040520,-118.307290,3.000000,1.000000,2.000000,15.000000,4.880000,2.000000
75%,5.044960e+07,2.107527e+08,8.000000,34.099525,-118.082105,6.000000,2.000000,3.000000,54.000000,5.000000,7.000000
max,6.420077e+17,4.627862e+08,3322.000000,34.822060,-117.627790,16.000000,17.000000,30.000000,1512.000000,5.000000,532.000000


### Visualize Attributes

In [17]:
#Visualize the location of the AirBNB properties on a map
airBNBMap = px.density_mapbox(df, lat='latitude', lon='longitude', radius=1, mapbox_style="open-street-map")
airBNBMap.show()

### Explore Joint Attributes

### Explore Attributes and Class

### New Features

### Exceptional Work